<a href="https://colab.research.google.com/github/likw99/awesome-colab/blob/main/IOPaint_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IOPaint in Google Colab
#@markdown In order to use this colab:
#@markdown - Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken)
#@markdown - Select a model ([find documentation here](https://www.iopaint.com/models))
#@markdown - Select the T4 GPU accelerator at `Runtime > Change runtime type > Hardware accelerator`
#@markdown - Click `Runtime > Run all`
#@markdown - Follow the link to access the UI

#@markdown Note: beware that installing models for the first time will take some time.
ngrok_token = "" # @param {type:"string"}
model = "lama" # @param ["lama", "mat", "migan", "runwayml/stable-diffusion-inpainting", "Sanster/PowerPaint-V1-stable-diffusion-inpainting", "Uminosachi/realisticVisionV51_v51VAE-inpainting", "Sanster/anything-4.0-inpainting", "redstonehero/dreamshaper-inpainting", "Sanster/AnyText", "timbrooks/instruct-pix2pix", "Fantasy-Studio/Paint-by-Example"]
enable_interactive_seg = False # @param {type: "boolean"}
interactive_seg_device = "cpu" # @param ["cuda", "cpu"]
interactive_seg_model = "sam_hq_vit_b" # @param ["sam_hq_vit_b", "sam_hq_vit_l", "sam_hq_vit_h"]
enable_remove_bg = False # @param {type: "boolean"}
remove_bg_model = "briaai/RMBG-1.4" # @param ["u2net", "u2netp", "u2net_human_seg", "u2net_cloth_seg", "silueta", "isnet-general-use", "briaai/RMBG-1.4"]
enable_realesrgan = False # @param {type: "boolean"}
realesrgan_model = "realesr-general-x4v3" # @param ["realesr-general-x4v3", "RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B"]
realesrgan_device = "cpu" # @param ["cuda", "cpu"]
enable_gfpgan = False # @param {type: "boolean"}
gfpgan_device = "cpu" # @param ["cuda", "cpu"]
enable_restoreformer = False # @param {type: "boolean"}
restoreformer_device = "cpu" # @param ["cuda", "cpu"]

# Download and install iopaint and pyngrok
!pip install iopaint pyngrok > install_logs.txt
!iopaint install-plugins-packages > plugins_install_logs.txt

import os
import asyncio
from pyngrok import ngrok
import sys

os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

print("Remember, installing models for the first time will be very slow! Be patient.")

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

if not ngrok_token:
  print("Hey! You haven't set the Ngrok token. Without the token, the app won't start. Please paste your token in the input.")
  sys.exit("Ngrok token not set")

ngrok.set_auth_token(ngrok_token)
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    ngrok.disconnect(public_url)
url = ngrok.connect(addr="8000", bind_tls=True)
print(f"IOPaint public URL: {url}")
print("Please wait until the \"Application startup complete.\" message appears...")

cmds = [
    'iopaint', 'start', '--model', model, '--device', 'cuda', '--port', '8000',
    '--interactive-seg-device', interactive_seg_device,
    '--interactive-seg-model', interactive_seg_model,
    '--remove-bg-model', remove_bg_model,
    '--realesrgan-model', realesrgan_model,
    '--realesrgan-device', realesrgan_device,
    '--gfpgan-device', gfpgan_device,
    '--restoreformer-device', restoreformer_device
]

if enable_interactive_seg:
  cmds.append('--enable-interactive-seg')
if enable_remove_bg:
  cmds.append('--enable-remove-bg')
if enable_realesrgan:
  cmds.append('--enable-realesrgan')
if enable_gfpgan:
  cmds.append('--enable-gfpgan')
if enable_restoreformer:
  cmds.append('--enable-restoreformer')

await asyncio.gather(
    run_process(cmds)
)